In [ ]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
from tqdm import tqdm
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from VGG16 import VGG16
from ResNet101 import ResNet101
from Inception_V2 import Inception_V2
from Inception_V2_LSTM import Inception_V2_LSTM
from VGG16_LSTM import VGG16_LSTM
from ResNet101_LSTM import ResNet101_LSTM
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import logging

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./LOG/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)

In [ ]:
save_model_file_path = './save_model/{}_{}.{}'
with open('/mnt/2ndSSD/FirstImpressions_V2/Preprocess_data/Visual/15Frames/Full_frames/train_set.dat', "rb") as training_file:
    train_set_data = pickle.load(training_file)
train_set_data=train_set_data[:3000]
with open('/mnt/2ndSSD/FirstImpressions_V2/Preprocess_data/Visual/15Frames/Full_frames/valid_set.dat', "rb") as validation_file:
    validation_set_data = pickle.load(validation_file)
validation_set_data=validation_set_data[:1000]

In [ ]:
def reshape_to_expected_input(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    x0_list = []
    x1_list = []
    for i in range(0, len(dataset)):
        x0_list.append(dataset[i][0])
        x1_list.append(dataset[i][1])
    return (np.stack(x0_list), np.stack(x1_list))

In [ ]:
class ChalearnDataset(Dataset):
    def __init__(self,imagedata,tagdata,transform=None):
        self.imagedata=imagedata
        self.tagdata=tagdata
        self.transform = transform  
    def __len__(self):
        return len(self.imagedata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image_data=self.imagedata[idx]
        image_data=torch.FloatTensor(image_data)
        image_data=image_data.permute(0,3,1,2)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor(big_five_sorces)
        return image_data,big_five_sorces

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

In [ ]:
logger.debug('=============InceptionV2Train Strat!=============')
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

In [ ]:
batchsz = 5
num_workerssz = 5
lr = 0.001
epochs = 32
resnet_out_size = 512
lstm_input_size = resnet_out_size
lstm_hidden_size = 256
lstm_num_layers = 2
lstm_output_size = 5

In [ ]:
train_set_datas = reshape_to_expected_input(train_set_data)
val_set_datas = reshape_to_expected_input(validation_set_data)
train_dataset = ChalearnDataset(imagedata=train_set_datas[0],tagdata=train_set_datas[1],transform=transform)
val_dataset = ChalearnDataset(imagedata=val_set_datas[0],tagdata=val_set_datas[1],transform=transform)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsz, shuffle=True, num_workers=num_workerssz)

In [ ]:
model=VGG16.oneVGG16_0()
# model=ResNet101.resnet101(num_classes=5)
# model = Inception_V2.InceptionV2_0()
# model = Inception_V2_LSTM.inceptionv2_LSTM()
# model = VGG16_LSTM.VGG16_LSTM()
# model = ResNet101_LSTM.ResNetLSTM(resnet_out_size=resnet_out_size, lstm_input_size=lstm_input_size, lstm_hidden_size=lstm_hidden_size, lstm_num_layers=lstm_num_layers, lstm_output_size=lstm_output_size)
model.to(device)
criterion = torch.nn.L1Loss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

In [ ]:
start_epoch = 0
test_flage=False
if test_flage==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'))
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [ ]:
with torch.cuda.device(0):
    for i in range(epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        for image_data, big_five_data in tqdm(train_dataloader):
            image_data = image_data.to(device)
            big_five_data=big_five_data.permute(0,2,1)
            big_five_data=big_five_data.squeeze()
            big_five_data=big_five_data.to(device)
            optimizer.zero_grad()  
            hypothesis = model(image_data)  
            loss = criterion(hypothesis, big_five_data)  
            loss.backward()  
            optimizer.step()  
            train_avg_loss += loss  
        train_avg_loss=train_avg_loss/len(train_dataloader)
        with torch.no_grad():
            for image_data, big_five_data in tqdm(val_dataloader):
                image_data = image_data.to(device)
                big_five_data=big_five_data.permute(0,2,1)
                big_five_data=big_five_data.squeeze()
                big_five_data=big_five_data.to(device)
                hypothesis = model(image_data)
                val_loss = criterion(hypothesis, big_five_data)
                val_avg_loss += val_loss
            val_avg_loss=val_avg_loss/len(val_dataloader)
        torch.cuda.empty_cache()
        max_value=val_avg_loss
        start_epoch+=1
        if (i + 1) % 5 == 0:
            torch.save({
                    'epoch': i+1,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'loss': val_avg_loss,
                }, save_model_file_path.format('model',start_epoch,'pth'))
        logger.debug('Epoch: {} , 1-MAE: {:.4f} , train_loss:{:.4f} , val_loss:{:.4f} , MAX Value:{:.4f} , MAX Value Epoch:{}'.format(i+1, 1-val_avg_loss, train_avg_loss, val_avg_loss, max_value, start_epoch))